In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from pyrates.utility.grid_search import grid_search, linearize_grid
from pyrates.utility.genetic_algorithm import GeneticAlgorithmTemplate, CGSGeneticAlgorithmTemplate
%matplotlib inline
warnings.filterwarnings("ignore")

## Genetic Algorithm

A genetic algorithm (GA) is an optimization method that can be used to identify model parameters in a defined parameter space that invoke a target model behaviour.
Generally, a certain amount of model parametrizations (population) are evaluated based on a problem dependant cost function (fitness function).
Based on the quality (fitness) of each parametrization, the population is modified and updated, applying strategies that are common for a GA.
PyRates' `GenticAlgorithmTemplate` class implements the following concepts that are utilized to update a population;

#### Selection
The fittest members of a population will be forwarded to the offspring
<img src="img/Selection.png" width="300" />

#### Mutation
A new population members is created by stochastically altering the parmeters of a parent parametrization
<img src="img/Mutation.png" width="300" />

#### Crossover
A new population members is created by merging the paremters of two parent parametrizations
<img src="img/Crossover.png" width="300" />

Members of the updated population are then again evaluated regarding their fitness.
This process is repeated until a certain abortion criterion is reached (maximum number of iterations, iterations without change or certain target fitness is exceeded).

The `GenticAlgorithmTemplate` and the `CGSGeneticAlgorithmTemplate` are designed in a way, that a whole population of parameter sets can be evaluated by the `grid_search` function or the `ClusterGridSearch` class.

### Defining a fitness function

In the following example, we will use the genetic algorithm to identify values for the two model parameters $\eta_{pc}$ and $\eta_{iin}$ of the aforemention circuit template (cf. CGS_presentation.ipynb) that invoke a desired mean firing rate in the PC of 1.0.

To run a parameter search, first the fitness function needs to be defined that assigns a fitness to each parametrization of the current population.
This is done by deriving a child class from the `GeneticAlgorithmTemplate` and adapting its `eval_fitness` method.
The `GeneticAlgorithmTemplate` provides the `pop_to_grid` method that converts the current parameter population into a parameter grid that can be computed by `grid_search`/`ClusterGridSearch`
We then iterate over each model output, compute the mean firing rate of the PC and assign a fitness based on the difference of the current mean value and the target mean value to each parametrization.

In [2]:
class GeneticAlgorithm(GeneticAlgorithmTemplate):
    def eval_fitness(self, target: list, **kwargs):
        
        # Set of model parametrization provided by the genetic algorithm
        pop_grid = self.pop_to_grid()
        
        param_map = {'eta_op': {'vars': ['Op_e/eta'],
                                'nodes': ['PC']},
                     'eta_iin': {'vars': ['Op_i/eta'],
                                 'nodes': ['IIN']}}
        
        T = 5.
        dt = 1e-4
        results, result_map, t_ = grid_search(
            circuit_template="model_templates.montbrio.simple_montbrio.Net3",
            simulation_time=T,
            dt=dt,
            sampling_step_size=1e-3,
            inputs={"PC/Op_e/inp": np.ones((int(T/dt), 1))},
            outputs={'r': 'PC/Op_e/r'},
            param_grid=pop_grid,
            param_map=param_map,
            permute_grid=False,                           # Creates all permutations of the param_grid values if set to True
            init_kwargs={                                 # Additional keyword arguments for the compute graph
                'backend': 'numpy',                       # Can be either 'numpy' or 'tensorflow'
                'solver': 'euler'                         # Solver for the differential equation approximation.
            },
            profile='t',                                  # Automated runtime tracking
            njit=True,                                    # Use Numba's njit compiler
            parallel=False
        )
        
        for i, (idx, data) in enumerate(results.iteritems()):
            current = np.mean(data)
            self.pop.loc[i, 'fitness'] = 1 / (1 + np.abs(target[0] - current))
            self.pop.loc[i, 'mean'] = current

Before the parameter search is started, the parameter space has to be defined from which the initial population will be drawn.
This is done in form of a dictionary. 
N defines how many samples will be drawn from the respective parameter space. The inirial population is created by permuting all sampled genes.
Sigma defines the range in which a new parameter will be sampled during a mutation.

If a drop save directory is passed to the genetic algorithm, it has to be ensured that this directory exists before the computation is started.
This directory will contain all population memebers that have been dropped from the population due to stagnation or convergence.

After a class instance is create, it's `run` method can be called, excepting various meta parameters for the fitting process as function arguments.  
<b> max_iter: </b> Number of performed iterations before the computation is stopped  
<b> min_fit: </b> Minimum fitness that needs to be reached for a population member to become the winner  
<b> target: </b> Desired target value (cf. fitness function)  
<b> n_winners: </b> Number of fittest members that are transferred to the offspring  
<b> n_parent_pairs: </b> Number of parent pairs that create a child for the offspring  
<b> n_new: </b> Number of parametrizations that are drawn from the intial gene pool and added to the offspring  
<b> sigma_adapt: </b> Ratio by which sigma will adept during a mutation (cf. Beyer1995)  
<b> max_stagnation_steps: </b> Number of iterations without a change in fitness before the fittest member of the current population is dropped    
<b> stagnation_decimals: </b> Number of decimal places to take into account for the stagnation check  
<b> new_pop_on_drop: </b> If True, a whole new population will be created once the computation stagnates  
<b> candidate_save: </b> Path to safe the current fittest member to  
<b> drop_save: </b> Path to safe the population member that is dropped out of the population  
<b> enforce_max_iter: </b> If True, all desired iterations will be performed, regarding any abortion or convergence criterion

In [3]:
pop_genes = {
    'eta_op': {
        'min': -20,
        'max': 20,
        'N': 10,
        'sigma': 0.5
    },
    'eta_iin': {
        'min': -20,
        'max': 20,
        'N': 10,
        'sigma': 0.5
    }
}

# Save drop outs of the population
drop_save_dir = f'~/Documents/PopulationDrops/'
os.makedirs(drop_save_dir, exist_ok=True)

ga = GeneticAlgorithm()

winner = ga.run(initial_gene_pool=pop_genes,
                max_iter=3,
                min_fit=0.98,
                target=[5.0],
                n_winners=10,
                n_parent_pairs=20,
                n_new=10,
                sigma_adapt=0.0,  # (cf. Beyer1995)
                max_stagnation_steps=20,
                stagnation_decimals=3,
                new_pop_on_drop=True,
                candidate_save=f'~/Documents/GeneticCGSCandidate.h5',
                drop_save=drop_save_dir,
                enforce_max_iter=False,
                )

***STARTING GENETIC ALGORITHM***

ITERATION: 0
building the compute graph...
Preparing the simulation...
Running the simulation...
5.0s of backend behavior were simulated in 6.838052749633789 s given a simulation resolution of 0.0001 s.
Currently fittest genes:
eta_op: 20.0  [min: -20, max: 20]
eta_iin: -20.0  [min: -20, max: 20]
fitness: 0.29309 
sigma: [0.5, 0.5, ] 
mean: 2.58805 
Target: [5.0]
Generating offspring
Updating population

ITERATION: 1
building the compute graph...
Preparing the simulation...
Running the simulation...
5.0s of backend behavior were simulated in 7.056836128234863 s given a simulation resolution of 0.0001 s.
Currently fittest genes:
eta_op: 19.6592  [min: -20, max: 20]
eta_iin: -19.95976  [min: -20, max: 20]
fitness: 0.29362 
sigma: [0.5, 0.5, ] 
mean: 2.59427 
Target: [5.0]
Generating offspring
Updating population

ITERATION: 2
building the compute graph...
Preparing the simulation...
Running the simulation...
5.0s of backend behavior were simulated in 6.9

### Utilizing cluster computations

The `CGSGeneticAlgorithmTemplate` has a built-in ability to use the `ClusterGridSearch`.
It can be accessed using the `self.cgs` instance inside the fitness_function.

In the example below, we will utilize a CGS instance to compute the model output for each population member.
Additionally, the post processing (here calculating the mean firing rate of the PC) will be already performed on the workers.

In [4]:
class CGSGeneticAlgorithm(CGSGeneticAlgorithmTemplate):
    def eval_fitness(self, target: list, **kwargs):
        
        # Set of model parametrization provided by the genetic algorithm
        pop_grid = self.pop_to_grid()
        
        param_map = {'eta_op': {'vars': ['Op_e/eta'],
                                'nodes': ['PC']},
                     'eta_iin': {'vars': ['Op_i/eta'],
                                 'nodes': ['IIN']}}
        
        T = 5.
        dt = 1e-4
        result_file = self.cgs.run(
            circuit_template="model_templates.montbrio.simple_montbrio.Net3",
            simulation_time=T,
            dt=dt,
            sampling_step_size=1e-3,
            inputs={"PC/Op_e/inp": np.ones((int(T/dt), 1))},
            outputs={'r': 'PC/Op_e/r'},
            param_grid=pop_grid,
            param_map=param_map,
            chunk_size=50,
            worker_file=f'{os.getcwd()}/my_cgsga_worker.py',
            add_template_info=False,
            gs_kwargs={
                "init_kwargs": {
                    'backend': 'numpy',
                    'solver': 'midpoint'
                },
                "njit": True,
                "parallel": False
            },
            verbose=True
        )
        
        results = pd.read_hdf(result_file, key="Results/results")

        for i, (idx, data) in enumerate(results.iteritems()):
            current = float(data)
            self.pop.loc[i, 'fitness'] = 1 / (1 + np.abs(target[0] - current))
            self.pop.loc[i, 'mean'] = current

To have the desired post processing performed by the cluster computation, a customized cluster worker is required (cf. CGS_presentation.ipynb)

In [ ]:
# my_cgsga_worker.py
import numpy as np
from pyrates.utility.grid_search import ClusterWorkerTemplate


class MyWorker(ClusterWorkerTemplate):
    def worker_postprocessing(self, **worker_kwargs):
        
        # Pre-define index if only a single value is stored in each column
        self.processed_results.index = "mean"

        for idx, data in self.results.iteritems():
            self.processed_results.loc['mean', idx] = np.mean(data)


if __name__ == "__main__":
    cgs_worker = MyWorker()
    cgs_worker.worker_init()

We can now run a parameter search, using the cluster computation class.
For that purpose, the desired nodes and an optional compute directory has to be passed upon the instantiaion of the `CGSGeneticAlgorithm` class.

In [5]:
pop_genes = {
    'eta_op': {
        'min': -20,
        'max': 20,
        'N': 10,
        'sigma': 0.5
    },
    'eta_iin': {
        'min': -20,
        'max': 20,
        'N': 10,
        'sigma': 0.5
    }
}

drop_save_dir = f'~/Documents/PopulationDrops/'
os.makedirs(drop_save_dir, exist_ok=True)

cgsga = CGSGeneticAlgorithm(
    nodes=[
        'animals',
        'osttimor'
    ],
    compute_dir="~/Documents/GA_presentation"
)

winner = cgsga.run(initial_gene_pool=pop_genes,
                   max_iter=3,
                   min_fit=0.95,
                   target=[5.0],
                   n_winners=10,
                   n_parent_pairs=20,
                   n_new=10,
                   sigma_adapt=0.0,  # (cf. Beyer1995)
                   max_stagnation_steps=20,
                   stagnation_decimals=3,
                   new_pop_on_drop=True,
                   candidate_save=f'~/Documents/GeneticCGSCandidate.h5',
                   drop_save=drop_save_dir,
                   enforce_max_iter=False,
                   )

***NEW CLUSTER INSTANCE CREATED***
Compute ID: 181119-130737
Compute directory: /data/hu_salomon/Documents/CGSGA_Test
Global logfile: /data/hu_salomon/Documents/CGSGA_Test/Logs/181119-130737/Global_logfile.log

***CONNECTING TO NODES...***
'animals': Connection established!
CPU: Intel(R) Core(TM) i7-4770K CPU @ 3.50GHz
Cores: 8
CPU freq: 3900.0 MHz
Total memory: 15901 MByte

'osttimor': Connection established!
CPU: Intel(R) Core(TM) i7-2600K CPU @ 3.40GHz
Cores: 8
CPU freq: 3800.0 MHz
Total memory: 16023 MByte

Nodes connected. Elapsed time: 2.353 seconds
***STARTING GENETIC ALGORITHM***

ITERATION: 0

***PREPARING PARAMETER GRID***
Done! Elapsed time: 0.019 seconds

***CHECKING PARAMETER GRID AND MAP FOR CONSISTENCY***
All parameter grid keys found in parameter map
Done! Elapsed time: 0.000 seconds

***CREATING CONFIG FILE***
Done! Elapsed time: 0.458 seconds

***CREATING GLOBAL RESULT FILE***
Done. Elapsed time: 0.026 seconds

***STARTING CLUSTER COMPUTATION***
[T]'animals': Fetching